# Df 0 0 4_Cut Flow Report
This tutorial shows how to get information about the efficiency of the filters
applied




**Author:** Danilo Piparo  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 01:00 AM.</small></i>

In [1]:
using FourVector = ROOT::Math::XYZTVector;
using FourVectors = std::vector<FourVector>;
using CylFourVector = ROOT::Math::RhoEtaPhiVector;

  A simple helper function to fill a test tree: this makes the example
 stand-alone.
 

In [2]:
%%cpp -d
void fill_tree(const char *treeName, const char *fileName)
{
   ROOT::RDataFrame d(50);
   int i(0);
   d.Define("b1", [&i]() { return (double)i; })
      .Define("b2",
              [&i]() {
                 auto j = i * i;
                 ++i;
                 return j;
              })
      .Snapshot(treeName, fileName);
}

We prepare an input tree to run on

In [3]:
auto fileName = "df004_cutFlowReport.root";
auto treeName = "myTree";
fill_tree(treeName, fileName);

We read the tree from the file and create a rdataframe

In [4]:
ROOT::RDataFrame d(treeName, fileName, {"b1", "b2"});

 ## Define cuts and create the report
 Here we define two simple cuts

In [5]:
auto cut1 = [](double b1) { return b1 > 25.; };
auto cut2 = [](int b2) { return 0 == b2 % 2; };

An optional string parameter name can be passed to the filter method to create a named filter.
 Named filters work as usual, but also keep track of how many entries they accept and reject.

In [6]:
auto filtered1 = d.Filter(cut1, {"b1"}, "Cut1");
auto filtered2 = d.Filter(cut2, {"b2"}, "Cut2");

auto augmented1 = filtered2.Define("b3", [](double b1, int b2) { return b1 / b2; });
auto cut3 = [](double x) { return x < .5; };
auto filtered3 = augmented1.Filter(cut3, {"b3"}, "Cut3");

Statistics are retrieved through a call to the report method:
 when Report is called on the main RDataFrame object, it retrieves stats
 for all named filters declared up to that point.
 When called on a stored chain state (i.e. a chain/graph node), it
 retrieves stats for all named filters in the section of the chain between
 the main RDataFrame and that node (included).
 Stats are printed in the same order as named filters have been added to
 the graph, and refer to the latest event-loop that has been run using the
 relevant RDataFrame.

In [7]:
std::cout << "Cut3 stats:" << std::endl;
filtered3.Report()->Print();

IncrementalExecutor::executeFunction: symbol '_ZSt4moveIRSt10shared_ptrIN4ROOT6Detail3RDF7RFilterIZ16__cling_Un1Qu317PvE3$_0NS4_IZ16__cling_Un1Qu316S5_E3$_1NS3_12RLoopManagerEEEEEEEONSt16remove_referenceIT_E4typeEOSE_' unresolved while linking [cling interface function]!
You are probably missing the definition of std::remove_reference<std::shared_ptr<ROOT::Detail::RDF::RFilter<__cling_Un1Qu317(void*)::$_0, ROOT::Detail::RDF::RFilter<__cling_Un1Qu316(void*)::$_1, ROOT::Detail::RDF::RLoopManager> > >&>::type&& std::move<std::shared_ptr<ROOT::Detail::RDF::RFilter<__cling_Un1Qu317(void*)::$_0, ROOT::Detail::RDF::RFilter<__cling_Un1Qu316(void*)::$_1, ROOT::Detail::RDF::RLoopManager> > >&>(std::shared_ptr<ROOT::Detail::RDF::RFilter<__cling_Un1Qu317(void*)::$_0, ROOT::Detail::RDF::RFilter<__cling_Un1Qu316(void*)::$_1, ROOT::Detail::RDF::RLoopManager> > >&)
Maybe you need to load the corresponding shared library?


It is not only possible to print the information about cuts, but also to
 retrieve it to then use it programmatically.

In [8]:
std::cout << "All stats:" << std::endl;
auto allCutsReport = d.Report();
allCutsReport->Print();

All stats:
Cut1      : pass=24         all=50         -- eff=48.00 % cumulative eff=48.00 %
Cut2      : pass=25         all=50         -- eff=50.00 % cumulative eff=50.00 %
Cut3      : pass=23         all=25         -- eff=92.00 % cumulative eff=46.00 %


We can now loop on the cuts

In [9]:
std::cout << "Name\tAll\tPass\tEfficiency" << std::endl;
for (auto &&cutInfo : allCutsReport) {
   std::cout << cutInfo.GetName() << "\t" << cutInfo.GetAll() << "\t" << cutInfo.GetPass() << "\t"
             << cutInfo.GetEff() << " %" << std::endl;
}

Name	All	Pass	Efficiency
Cut1	50	24	48 %
Cut2	50	25	50 %
Cut3	25	23	92 %


Or get information about them individually

In [10]:
auto cutName = "Cut1";
auto cut = allCutsReport->At("Cut1");
std::cout << cutName << " efficiency is " << cut.GetEff() << " %" << std::endl;

Cut1 efficiency is 48 %


Draw all canvases 

In [11]:
gROOT->GetListOfCanvases()->Draw()